# Movie Reviews Polarity Scoring using a RNN

In [1]:
# movie review 
# a recurrent neural network for movie reviews classification / polarity measure
# 
# amir 2021 ...
#
from tensorflow.keras.preprocessing import text_dataset_from_directory
from tensorflow.strings import regex_replace
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout

In [2]:

def prepareData(dir):
  data = text_dataset_from_directory(dir)
  return data.map(
    lambda text, label: (regex_replace(text, '<br />', ' '), label),
  )

In [3]:
# Assumes you're in the root level of the dataset directory.
# If you aren't, you'll need to change the relative paths here.
train_data = prepareData('movie-reviews-dataset/train')
test_data = prepareData('movie-reviews-dataset/test')

for text_batch, label_batch in train_data.take(1):
  print(text_batch.numpy()[0])
  print(label_batch.numpy()[0]) # 0 = negative, 1 = positive


model = Sequential()

# ----- 1. INPUT
# We need this to use the TextVectorization layer next.
model.add(Input(shape=(1,), dtype="string"))

# ----- 2. TEXT VECTORIZATION
# This layer processes the input string and turns it into a sequence of
# max_len integers, each of which maps to a certain token.
max_tokens = 1000
max_len = 100
vectorize_layer = TextVectorization(
  # Max vocab size. Any words outside of the max_tokens most common ones
  # will be treated the same way: as "out of vocabulary" (OOV) tokens.
  max_tokens=max_tokens,
  # Output integer indices, one per string token
  output_mode="int",
  # Always pad or truncate to exactly this many tokens
  output_sequence_length=max_len,
)

# Call adapt(), which fits the TextVectorization layer to our text dataset.
# This is when the max_tokens most common words (i.e. the vocabulary) are selected.
train_texts = train_data.map(lambda text, label: text)
vectorize_layer.adapt(train_texts)

model.add(vectorize_layer)

# ----- 3. EMBEDDING
# This layer turns each integer (representing a token) from the previous layer
# an embedding. Note that we're using max_tokens + 1 here, since there's an
# out-of-vocabulary (OOV) token that gets added to the vocab.
model.add(Embedding(max_tokens + 1, 128))

# ----- 4. RECURRENT LAYER
model.add(LSTM(64))

# ----- 5. DENSE HIDDEN LAYER
model.add(Dense(64, activation="relu"))

# ----- 6. OUTPUT
model.add(Dense(1, activation="sigmoid"))


# Compile and train the model.
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(train_data, epochs=10, verbose=1)     # -- was 10

model.save_weights('rnn')

model.load_weights('rnn')

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
b"I took this out arbitrarily from the library the other night, having no idea of the film's cult, influence, or that it is currently being staged as a musical.(!) Most of the comments here are on target, it's moving, funny, sad, and yes, a tad exploitive despite the best intentions of the filmmakers. The expanded Chriterion edition is a must for anyone who loved it when it came out.   I think you can also see in little Edie the fall of a class that sort of disappeared, you can hear it in old films of Jackie O too; people just don't talk like that anymore. I think as a documentary, it would have been interesting to get more information about how the home fell into disrepute, Old Edie at least still seems aware of what's going on to a certain degree; couldn't She see the once spectacular home disintegrating?   Yet the film's subject is the life the two women have constructed for themselves now, a real li

In [4]:
# Try the model on our test dataset.
model.evaluate(test_data)

# Should print a very high score like 0.98.
print(model.predict([
  "i loved it! highly recommend it to anyone and everyone looking for a great movie to watch.",
]))

# Should print a very low score like 0.01.
print(model.predict([
  "this was awful! i hated it so much, nobody should watch this. the acting was terrible, the music was terrible, overall it was just bad.",
]))

782/782 [==============================] - 76s 96ms/step - loss: 0.5491 - accuracy: 0.7808
[[0.9795991]]
[[0.01131651]]


In [5]:
print(model.predict([
  "bad",
]))


[[0.01795468]]


In [6]:
print(model.predict([
  "I liked that.",
]))

[[0.9459926]]


In [7]:
print(model.predict([
  "such a nice movie.",
]))

[[0.94609725]]


In [8]:
print(model.predict([
  "I'm in love with that film.",
]))

[[0.8240361]]


In [14]:
print(model.predict([
  "I love this movie very much but the song was average",
]))

[[0.94364315]]
